In [14]:
import numpy as np

In [ ]:
import os
import ctypes
import sys


modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

# library path
intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path

intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

# # Python paths
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

In [48]:
import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

# Import our utility functions
from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


In [49]:
import uproot

In [72]:
import pandas as pd
import numpy as np
import uproot

def angle(v1: np.array, v2: np.array) -> float:
    """ Calculates the angle between two vectors in radians

    Parameters
    ----------
    v1: np.array
        vector 1
    v2: np.array
        vector 2

    Returns
    -------
    angle: float
        The calculates angle in radians
    """
    angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))
    return angle

def p2azimuthAndzenith(p: np.array):
    """ converts a momentum vector to azimuth and zenith angles

    Parameters
    ----------
    p: np.array
        The 3d momentum

    Returns
    -------
    float, float:
        The azimuth and zenith angles in radians.c
    """
    azimuth = angle(p[:2], np.array([0, 1]))
    zenith = angle(p[1:], np.array([0., 1]))
    return azimuth, zenith

def genie_parser(events)->pd.DataFrame:
    """ function to fetch the relevant information from genie events (in rootracker format)

    Parameters
    ----------
    events : dict
        The genie events

    Returns
    -------
    pd.DataFrame
        Data frame containing the relevant information
    """
    dic = {}
    dic['event_description'] = events['EvtCode/fString'].array(library='np')  # String describing the event
    dic['event_id'] = events['EvtNum'].array(library="np")  # Number of the event
    dic['event_children'] = events['StdHepN'].array(library="np")  # NUmber of the generated sub-particles
    dic['event_prob'] = events['EvtProb'].array(library="np")  # Probability of the event
    dic['event_xsec'] = events['EvtXSec'].array(library="np")  # Total xsec of the event
    dic['event_dxsec'] = events['EvtDXSec'].array(library="np")  # Total DXsec of the event
    dic['event_pdg_id'] = events['StdHepPdg'].array(library="np")  # PDG ids of all produced particles
    dic['event_momenta'] = events['StdHepP4'].array(library="np")  # Momenta of the particles
    tmp = events['EvtVtx'].array(library="np")  # Position of the particles
    dic['event_vertex'] = [np.array(vtx) for vtx in tmp]
    dic['event_coords'] = events['StdHepX4'].array(library="np")  # Position of the particles
    dic['event_weight'] = events['EvtWght'].array(library='np')  # Weight of the events
    tmp = events['StdHepStatus'].array(library="np")  # Status of the particle
    # Converting the codes
    particle_dic = {
        0: 'initial',
        1: 'final',
        2: 'intermediate',
        3: 'decayed',
        11: 'nucleon target',
        12: 'DIS pre-frag hadronic state',
        13: 'resonance',
        14: 'hadron in nucleus',
        15: 'final nuclear',
        16: 'nucleon cluster target',
    }
    new_arr = np.array([[
        particle_dic[particle] for particle in event
    ] for event in tmp], dtype=object)
    dic['event_status'] = new_arr
    return pd.DataFrame.from_dict(dic)

def final_parser(parsed_events: pd.DataFrame)->pd.DataFrame:
    """ fetches the final states

    Parameters
    ----------
    parsed_events : pd.DataFrame
        The parsed events

    Returns
    -------
    pd.DataFrame
        The inital + final state info
    """
    inital_energies_inj = np.array([event[0][3] for event in parsed_events['event_momenta']])
    inital_momenta_inj = [np.array(event[0][:3]) for event in parsed_events['event_momenta']]
    inital_energies_target = np.array([event[1][3] for event in parsed_events['event_momenta']])
    inital_id_inj = np.array([event[0] for event in parsed_events['event_pdg_id']])
    inital_id_target = np.array([event[1] for event in parsed_events['event_pdg_id']])
    final_ids = np.array([np.where(event == np.array('final'), True, False) for event in parsed_events['event_status']], dtype=object)
    children_ids = np.array([
        event[final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_pdg_id'])
    ], dtype=object)
    children_energy = np.array([
        event[:, 3][final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_momenta'])
    ], dtype=object)
    children_momenta = np.array([
        event[:, :3][final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_momenta'])
    ], dtype=object)
    final_ids = np.array([np.where(event == np.array('final nuclear'), True, False) for event in parsed_events['event_status']], dtype=object)
    children_nuc_ids = np.array([
        event[final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_pdg_id'])
    ], dtype=object)
    children_nuc_energy = np.array([
        event[:, 3][final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_momenta'])
    ], dtype=object)
    dic = {}
    dic['event_descr'] = parsed_events['event_description']
    dic['event_xsec'] = parsed_events['event_xsec']
    dic['event_dxsec'] =  parsed_events['event_dxsec']
    dic['event_vertex'] = parsed_events.event_vertex
    dic['init_inj_e'] = inital_energies_inj
    dic['init_inj_p'] = inital_momenta_inj
    dic['init_target_e'] = inital_energies_target
    dic['init_inj_id'] = inital_id_inj
    dic['init_target_id'] = inital_id_target
    dic['final_ids'] = children_ids
    dic['final_e'] = children_energy
    dic['final_p'] = children_momenta
    dic['final_nuc_ids'] = children_nuc_ids
    dic['final_nuc_e'] = children_nuc_energy
    dic['p4'] = np.array([event for event in parsed_events['event_momenta']], dtype='object')
    return pd.DataFrame.from_dict(dic)

def genie2prometheus(parsed_events: pd.DataFrame):
    """ reformats parsed GENIE events into a usable format for PROMETHEUS
    NOTES: Create a standardized scheme function. This could then be used as an interface to PROMETHEUS
    for any injector. E.g. a user would only need to create a function to translate their injector output to the scheme format.

    Parameters
    ----------
    parsed_events: pd.DataFrame
        Dataframe object containing all the relevant (and additional) information needed by PROMETHEUS

    Returns
    -------
    particles: pd.DataFrame
        Fromatted data set with values which can be used directly.
    injection: pd.Dataframe
        The injected particle in the same format
    """
    # TODO: Use a more elegant definition to couple to the particle class
    primaries = {}
    event_set  = {}
    for index, event in parsed_events.iterrows():
        if 'CC' in event.event_descr:
            event_type = 'CC'
        elif 'NC' in event.event_descr:
            event_type = 'NC'
        else:
            event_type = 'other'
        azimuth, zenith =  p2azimuthAndzenith(event.init_inj_p)
        primary_particle = {
            "primary": True,  # If this thing is a primary particle
            "e": event.init_inj_e,  # The energy in GeV,  # NOTE: Should this be kinetic energy?
            "pdg_code": event.init_inj_id,  # The PDG id
            "interaction": event_type,  # Should we use ints to define interactions or strings?
            "theta": zenith,  # It's zenith angle
            "phi": azimuth,  # It's azimuth angle
            "bjorken_x": -1,  # Bjorken x variable
            "bjorken_y": -1,  # Bjorken y variable
            "pos_x": event.event_vertex[0],  # position x (in detector coordinates)
            "pos_y": event.event_vertex[1],  # position y (in detector coordinates)
            "pos_z": event.event_vertex[2],  # position z (in detector coordinates)
            "position": event.event_vertex[:3],  # 3d position
            "column_depth": -1,  # Column depth where the interaction happened
            'custom_info': event.event_descr  # Additional information the user can add as a string. # TODO: This should be handed to the propagators
        }
        # TODO: Optimize
        angles = np.array([p2azimuthAndzenith(p) for p in event.final_p])
        particles = {
            "primary": np.array(np.zeros(len(event.final_ids)), dtype=bool),  # If this thing is a primary particle
            "e": event.final_e,  # The energy in GeV,  # NOTE: Should this be kinetic energy?
            "pdg_code": event.final_ids,  # The PDG id
            "interaction": np.array([event_type for _ in range(len(event.final_ids))]),  # Should we use ints to define interactions or strings?
            "theta": angles[:, 1],  # It's zenith angle
            "phi": angles[:, 0],  # It's azimuth angle
            "bjorken_x": np.ones(len(event.final_ids)) * -1,  # Bjorken x variable
            "bjorken_y": np.ones(len(event.final_ids)) * -1,  # Bjorken y variable
            "pos_x": np.ones(len(event.final_ids)) * event.event_vertex[0],  # position x (in detector coordinates)
            "pos_y": np.ones(len(event.final_ids)) * event.event_vertex[1],  # position y (in detector coordinates)
            "pos_z": np.ones(len(event.final_ids)) * event.event_vertex[2],  # position z (in detector coordinates)
            "position": np.array([event.event_vertex[:3] for _ in range(len(event.final_ids))]),  # 3d position
            "column_depth": np.ones(len(event.final_ids)) * -1,  # Column depth where the interaction happened
            'custom_info': np.array(['child' for _ in range(len(event.final_ids))])  # Additional information the user can add as a string. # TODO: This should be handed to the propagators
        }
        event_set[index] = particles
        primaries[index] = primary_particle
    return pd.DataFrame.from_dict(event_set, orient='index'), pd.DataFrame.from_dict(primaries, orient='index')

def parse_and_convert_genie2(root_file_path, inject_in_cylinder=False, rotate_particles = False):
    """Combined function to load, parse and convert GENIE events to Prometheus format
    
    Parameters
    ----------
    root_file_path : str
        Path to the root file containing GENIE events
        
    Returns
    -------
    tuple
        (prometheus_set, primary_set) - DataFrames in Prometheus format
    """
    with uproot.open(root_file_path) as file:
        events = file['gRooTracker']
        parsed_events = genie_parser(events)
        final_parsed = final_parser(parsed_events)
    p1,p2 = genie2prometheus(final_parsed)
    return p1, p2, parsed_events, final_parsed

In [ ]:
gtrac_file = "/groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root"

prometheus_set, primary_set,parsed_events, final_parsed = parse_and_convert_genie2(gtrac_file)

In [ ]:
primary_set.iloc[99950]

In [ ]:
prometheus_set.iloc[99950]

In [ ]:
parsed_events.iloc[99950]

In [ ]:
final_parsed.iloc[99950]

In [ ]:
prometheus_set.iloc[99950]['pdg_code']

In [ ]:
parsed_events

In [ ]:
final_parsed

In [ ]:
parsed_events.iloc[99950]

In [ ]:
final_parsed.iloc[99950]

In [ ]:
final_parsed.iloc[99950]['event_descr']

In [ ]:
final_parsed.columns

In [ ]:
parsed_events.columns

In [91]:
event_xsec = np.array(final_parsed['event_xsec'])
event_dxsec = np.array(final_parsed['event_dxsec'])
event_e = np.array(final_parsed['init_inj_e'])

In [ ]:
final_parsed

In [ ]:
import matplotlib.pyplot as plt
e_bins = np.linspace(10, 60, 51)
plt.figure(figsize=(12,8))
plt.hist(event_e, edgecolor='black', bins = e_bins)
plt.xlabel('energy (GeV)')
plt.ylabel('count')
plt.plot()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8))
plt.scatter(event_e, event_xsec, s=5)
plt.xlabel('energy (GeV)')
plt.ylabel('event_xsec')
plt.plot()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the energy bins (same as your histogram)
e_bins = np.linspace(10, 60, 51)
bin_centers = (e_bins[:-1] + e_bins[1:]) / 2  # Calculate bin centers for plotting

# Group the event_xsec by energy bins and calculate mean per bin
# First, digitize the energy values to assign each to a bin
bin_indices = np.digitize(event_e, e_bins) - 1  # -1 because digitize returns bin indices starting from 1
bin_indices = np.clip(bin_indices, 0, len(e_bins)-2)  # Ensure indices are within valid range

# Calculate mean event_xsec for each bin
mean_xsec_per_bin = []
for i in range(len(e_bins)-1):
    # Find events in this bin
    mask = (bin_indices == i)
    if np.sum(mask) > 0:  # If bin has events
        mean_xsec = np.mean(event_xsec[mask])
    else:
        mean_xsec = 0  # or np.nan if you prefer
    mean_xsec_per_bin.append(mean_xsec)

# Create two subplots - one for original scatter, one for binned means
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

# Original scatter plot
ax1.scatter(event_e, event_xsec, s=5, alpha=0.5)
ax1.set_xlabel('Energy (GeV)')
ax1.set_ylabel('event_xsec')
ax1.set_title('Original Scatter Plot')

# Binned means plot
ax2.bar(bin_centers, mean_xsec_per_bin, width=(e_bins[1]-e_bins[0]), 
       align='center', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Energy (GeV)')
ax2.set_ylabel('Mean event_xsec')
ax2.set_title('Mean Cross-section per Energy Bin')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Paths to your CSV files
csv_dir = "/groups/icecube/jackp/tmp/"
o16_files = {
    'tot_cc': f"{csv_dir}o16_tot_cc.csv",
    'tot_nc': f"{csv_dir}o16_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}o16_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}o16_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}o16_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}o16_tot_nc_n.csv"
}

h1_files = {
    'tot_cc': f"{csv_dir}h1_tot_cc.csv",
    'tot_nc': f"{csv_dir}h1_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}h1_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}h1_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}h1_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}h1_tot_nc_n.csv"
}

# Load data from CSV files
o16_data = {}
for name, file in o16_files.items():
    try:
        df = pd.read_csv(file)
        o16_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

h1_data = {}
for name, file in h1_files.items():
    try:
        df = pd.read_csv(file)
        h1_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Load your event data
# You'll need to replace these with your actual event data arrays
# For example:
# event_e = your_event_energies
# event_xsec = your_event_cross_sections

# Create Figure 1: Just the cross-sections
plt.figure(figsize=(12, 8))

# Plot styles for different cross-section types
line_styles = {
    'tot_cc': {'style': '-', 'label': 'CC', 'color': 'b', 'width': 2},
    'tot_nc': {'style': '--', 'label': 'NC', 'color': 'b', 'width': 2},
    'tot_cc_p': {'style': ':', 'label': 'CC p', 'color': 'b', 'width': 1},
    'tot_cc_n': {'style': '-.', 'label': 'CC n', 'color': 'b', 'width': 1}
}

# Plot O16 cross-sections
for name, data in o16_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color=style['color'], linewidth=style['width'], 
                label=f'O16 {style["label"]}')

# Plot H1 cross-sections
for name, data in h1_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color='g', linewidth=style['width'], 
                label=f'H1 {style["label"]}')

# Calculate H2O cross-section for CC and NC
water_energies = np.linspace(10, 60, 100)
h2o_data = {}

for xsec_type in ['tot_cc', 'tot_nc']:
    if xsec_type in o16_data and xsec_type in h1_data:
        # Interpolate O16
        o16_interp = np.interp(water_energies, 
                               o16_data[xsec_type]['energy'], 
                               o16_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Interpolate H1
        h1_interp = np.interp(water_energies, 
                               h1_data[xsec_type]['energy'], 
                               h1_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Water = 1 oxygen + 2 hydrogen
        h2o_xsec = o16_interp + 2 * h1_interp
        
        # Store for later use
        h2o_data[xsec_type] = {
            'energy': water_energies,
            'xsec': h2o_xsec
        }
        
        # Plot H2O cross-section
        style = line_styles[xsec_type]
        plt.plot(water_energies, h2o_xsec, 
                style['style'], color='r', linewidth=3, 
                label=f'H2O {style["label"]}')

# Finalize the plot
plt.xlabel('Energy (GeV)')
plt.ylabel('Cross-section (10^-38 cm^2)')
plt.title('GENIE Cross-sections')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(10, 60)
plt.show()

# If you have event data, create Figure 2: Comparison with event data
if 'event_e' in globals() and 'event_xsec' in globals():
    # Create binned data
    e_bins = np.linspace(10, 60, 51)
    bin_centers = (e_bins[:-1] + e_bins[1:]) / 2
    bin_indices = np.digitize(event_e, e_bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(e_bins)-2)
    
    mean_xsec_per_bin = []
    events_per_bin = []
    for i in range(len(e_bins)-1):
        mask = (bin_indices == i)
        count = np.sum(mask)
        events_per_bin.append(count)
        if count > 0:
            mean_xsec = np.mean(event_xsec[mask])
        else:
            mean_xsec = 0
        mean_xsec_per_bin.append(mean_xsec)
    
    # Create a figure for comparison
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))
    
    # Plot cross-sections
    ax1.bar(bin_centers, mean_xsec_per_bin, width=(e_bins[1]-e_bins[0]),
            align='center', alpha=0.5, edgecolor='black', label='Event XSec')
    
    # Create separate y-axis for reference data
    ax1_ref = ax1.twinx()
    
    # Interpolate H2O CC cross-section to bin centers
    h2o_cc_interp = np.interp(bin_centers, 
                             h2o_data['tot_cc']['energy'], 
                             h2o_data['tot_cc']['xsec'], 
                             left=0, right=0)
    
    ax1_ref.plot(bin_centers, h2o_cc_interp, 'r-', linewidth=2, label='H2O CC')
    
    ax1.set_xlabel('Energy (GeV)')
    ax1.set_ylabel('Event XSec', color='blue')
    ax1_ref.set_ylabel('ROOT XSec (10^-38 cm^2)', color='red')
    
    ax1.legend(loc='upper left')
    ax1_ref.legend(loc='upper right')
    ax1.set_title('Comparison of Event XSec and Reference Cross-section Values')
    ax1.grid(True, alpha=0.3)
    
    # Plot event distribution
    ax2.bar(bin_centers, events_per_bin, width=(e_bins[1]-e_bins[0]),
            align='center', alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Energy (GeV)')
    ax2.set_ylabel('Number of Events')
    ax2.set_title('Event Count per Energy Bin')
    ax2.grid(True, alpha=0.3)
    
    # Plot expected distribution based on cross-section
    if np.sum(events_per_bin) > 0 and np.sum(h2o_cc_interp) > 0:
        # Scale to match total event count
        scaling = np.sum(events_per_bin) / np.sum(h2o_cc_interp)
        expected_events = h2o_cc_interp * scaling
        
        ax2.plot(bin_centers, expected_events, 'r--', linewidth=2, 
                 label='Expected from H2O CC (normalized)')
        ax2.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No event data available for comparison.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Paths to your CSV files
csv_dir = "/groups/icecube/jackp/tmp/"
o16_files = {
    'tot_cc': f"{csv_dir}o16_tot_cc.csv",
    'tot_nc': f"{csv_dir}o16_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}o16_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}o16_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}o16_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}o16_tot_nc_n.csv"
}

h1_files = {
    'tot_cc': f"{csv_dir}h1_tot_cc.csv",
    'tot_nc': f"{csv_dir}h1_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}h1_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}h1_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}h1_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}h1_tot_nc_n.csv"
}

# Load data from CSV files
o16_data = {}
for name, file in o16_files.items():
    try:
        df = pd.read_csv(file)
        o16_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

h1_data = {}
for name, file in h1_files.items():
    try:
        df = pd.read_csv(file)
        h1_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Assume we're working with your event data
# Replace these with your actual event data
# event_e = your_energy_array
# event_xsec = your_xsec_array

# Create Figure 1: Just the cross-sections
plt.figure(figsize=(12, 8))

# Plot styles for different cross-section types
line_styles = {
    'tot_cc': {'style': '-', 'label': 'CC', 'color': 'b', 'width': 2},
    'tot_nc': {'style': '--', 'label': 'NC', 'color': 'b', 'width': 2},
    'tot_cc_p': {'style': ':', 'label': 'CC p', 'color': 'b', 'width': 1},
    'tot_cc_n': {'style': '-.', 'label': 'CC n', 'color': 'b', 'width': 1}
}

# Plot O16 cross-sections
for name, data in o16_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color=style['color'], linewidth=style['width'], 
                label=f'O16 {style["label"]}')

# Plot H1 cross-sections
for name, data in h1_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color='g', linewidth=style['width'], 
                label=f'H1 {style["label"]}')

# Calculate H2O cross-section for CC and NC
water_energies = np.linspace(10, 60, 100)
h2o_data = {}

for xsec_type in ['tot_cc', 'tot_nc']:
    if xsec_type in o16_data and xsec_type in h1_data:
        # Interpolate O16
        o16_interp = np.interp(water_energies, 
                               o16_data[xsec_type]['energy'], 
                               o16_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Interpolate H1
        h1_interp = np.interp(water_energies, 
                               h1_data[xsec_type]['energy'], 
                               h1_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Water = 1 oxygen + 2 hydrogen
        h2o_xsec = o16_interp + 2 * h1_interp
        
        # Store for later use
        h2o_data[xsec_type] = {
            'energy': water_energies,
            'xsec': h2o_xsec
        }
        
        # Plot H2O cross-section
        style = line_styles[xsec_type]
        plt.plot(water_energies, h2o_xsec, 
                style['style'], color='r', linewidth=3, 
                label=f'H2O {style["label"]}')

# Calculate H2O total cross-section (CC + NC)
if 'tot_cc' in h2o_data and 'tot_nc' in h2o_data:
    h2o_total_xsec = h2o_data['tot_cc']['xsec'] + h2o_data['tot_nc']['xsec']
    h2o_data['tot'] = {
        'energy': water_energies,
        'xsec': h2o_total_xsec
    }
    plt.plot(water_energies, h2o_total_xsec, 
            '-', color='r', linewidth=4, 
            label=f'H2O Total')

# Finalize the plot
plt.xlabel('Energy (GeV)')
plt.ylabel('Cross-section (10^-38 cm^2)')
plt.title('GENIE Cross-sections')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(10, 60)
plt.show()

# If you have event data, create Figure 2: Comparison with event data
if 'event_e' in globals() and 'event_xsec' in globals():
    # Create binned data
    e_bins = np.linspace(10, 60, 51)
    bin_centers = (e_bins[:-1] + e_bins[1:]) / 2
    bin_indices = np.digitize(event_e, e_bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(e_bins)-2)
    
    mean_xsec_per_bin = []
    events_per_bin = []
    for i in range(len(e_bins)-1):
        mask = (bin_indices == i)
        count = np.sum(mask)
        events_per_bin.append(count)
        if count > 0:
            mean_xsec = np.mean(event_xsec[mask])
        else:
            mean_xsec = 0
        mean_xsec_per_bin.append(mean_xsec)
    
    # Create a figure with three subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 18))
    
    # Plot 1: Original scatter plot
    ax1.scatter(event_e, event_xsec, s=5, alpha=0.5)
    ax1.set_xlabel('Energy (GeV)')
    ax1.set_ylabel('Event XSec')
    ax1.set_title('Original Event Cross-sections Scatter Plot')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Binned means with reference cross-sections
    ax2.bar(bin_centers, mean_xsec_per_bin, width=(e_bins[1]-e_bins[0]),
            align='center', alpha=0.5, edgecolor='black', label='Event XSec')
    
    # Create separate y-axis for reference data
    ax2_ref = ax2.twinx()
    
    # Interpolate H2O cross-sections to bin centers
    xsec_types = ['tot_cc', 'tot_nc', 'tot']
    styles = ['-', '--', '-.']
    labels = ['CC', 'NC', 'Total']
    
    for i, xsec_type in enumerate(xsec_types):
        if xsec_type in h2o_data:
            h2o_interp = np.interp(bin_centers, 
                                 h2o_data[xsec_type]['energy'], 
                                 h2o_data[xsec_type]['xsec'], 
                                 left=0, right=0)
            
            ax2_ref.plot(bin_centers, h2o_interp, 
                      styles[i], color='r', linewidth=2, 
                      label=f'H2O {labels[i]}')
    
    ax2.set_xlabel('Energy (GeV)')
    ax2.set_ylabel('Event XSec', color='blue')
    ax2_ref.set_ylabel('Reference XSec (10^-38 cm^2)', color='red')
    
    ax2.legend(loc='upper left')
    ax2_ref.legend(loc='upper right')
    ax2.set_title('Comparison of Mean Event XSec and Reference Cross-sections')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Event distribution with expected distribution
    ax3.bar(bin_centers, events_per_bin, width=(e_bins[1]-e_bins[0]),
            align='center', alpha=0.7, edgecolor='black')
    ax3.set_xlabel('Energy (GeV)')
    ax3.set_ylabel('Number of Events')
    ax3.set_title('Event Count per Energy Bin')
    ax3.grid(True, alpha=0.3)
    
    # Calculate expected event distribution based on cross-section and E^-2 flux
    if np.sum(events_per_bin) > 0:
        for i, xsec_type in enumerate(xsec_types):
            if xsec_type in h2o_data:
                # Interpolate cross-section to bin centers
                h2o_interp = np.interp(bin_centers, 
                                     h2o_data[xsec_type]['energy'], 
                                     h2o_data[xsec_type]['xsec'], 
                                     left=0, right=0)
                
                # Apply E^-2 flux weighting
                flux_weight = 1.0 / (bin_centers ** 2)
                expected_distribution = h2o_interp * flux_weight
                
                # Scale to match total event count
                scaling = np.sum(events_per_bin) / np.sum(expected_distribution)
                expected_events = expected_distribution * scaling
                
                ax3.plot(bin_centers, expected_events, 
                       styles[i], color='r', linewidth=2, 
                       label=f'Expected from H2O {labels[i]} (E^-2 weighted)')
        
        ax3.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No event data available for comparison.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Paths to your CSV files
csv_dir = "/groups/icecube/jackp/tmp/"
o16_files = {
    'tot_cc': f"{csv_dir}o16_tot_cc.csv",
    'tot_nc': f"{csv_dir}o16_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}o16_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}o16_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}o16_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}o16_tot_nc_n.csv"
}

h1_files = {
    'tot_cc': f"{csv_dir}h1_tot_cc.csv",
    'tot_nc': f"{csv_dir}h1_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}h1_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}h1_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}h1_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}h1_tot_nc_n.csv"
}

# Load data from CSV files
o16_data = {}
for name, file in o16_files.items():
    try:
        df = pd.read_csv(file)
        o16_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

h1_data = {}
for name, file in h1_files.items():
    try:
        df = pd.read_csv(file)
        h1_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Assume we're working with your event data


# Create Figure 1: Just the cross-sections+
plt.figure(figsize=(18, 12))

# Plot styles for different cross-section types
line_styles = {
    'tot_cc': {'style': '-', 'label': 'CC', 'color': 'b', 'width': 2},
    'tot_nc': {'style': '--', 'label': 'NC', 'color': 'b', 'width': 2},
    'tot_cc_p': {'style': ':', 'label': 'CC p', 'color': 'b', 'width': 1},
    'tot_cc_n': {'style': '-.', 'label': 'CC n', 'color': 'b', 'width': 1}
}

# Plot O16 cross-sections
for name, data in o16_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color=style['color'], linewidth=style['width'], 
                label=f'O16 {style["label"]}')

# Plot H1 cross-sections
for name, data in h1_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color='g', linewidth=style['width'], 
                label=f'H1 {style["label"]}')

# Calculate H2O cross-section for CC and NC
water_energies = np.linspace(10, 60, 100)
h2o_data = {}

for xsec_type in ['tot_cc', 'tot_nc']:
    if xsec_type in o16_data and xsec_type in h1_data:
        # Interpolate O16
        o16_interp = np.interp(water_energies, 
                               o16_data[xsec_type]['energy'], 
                               o16_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Interpolate H1
        h1_interp = np.interp(water_energies, 
                               h1_data[xsec_type]['energy'], 
                               h1_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Water = 1 oxygen + 2 hydrogen
        h2o_xsec = o16_interp + 2 * h1_interp
        
        # Store for later use
        h2o_data[xsec_type] = {
            'energy': water_energies,
            'xsec': h2o_xsec
        }
        
        # Plot H2O cross-section
        style = line_styles[xsec_type]
        # plt.plot(water_energies, h2o_xsec, 
        #         style['style'], color='r', linewidth=1, 
        #         label=f'H2O {style["label"]}')

# Calculate H2O total cross-section (CC + NC)
if 'tot_cc' in h2o_data and 'tot_nc' in h2o_data:
    h2o_total_xsec = h2o_data['tot_cc']['xsec'] + h2o_data['tot_nc']['xsec']
    h2o_data['tot'] = {
        'energy': water_energies,
        'xsec': h2o_total_xsec
    }
    # plt.plot(water_energies, h2o_total_xsec, 
    #         '-', color='r', linewidth=4, 
    #         label=f'H2O Total')
# Original scatter plot
plt.scatter(event_e, event_xsec,color='red', s=8, alpha=0.01)

# Finalize the plot
plt.xlabel('Energy (GeV)')
plt.ylabel('Cross-section (10^-38 cm^2)')
plt.title('GENIE Cross-sections')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(10, 60)
plt.ylim(0,900)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Paths to your CSV files
csv_dir = "/groups/icecube/jackp/tmp/"
o16_files = {
    'tot_cc': f"{csv_dir}o16_tot_cc.csv",
    'tot_nc': f"{csv_dir}o16_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}o16_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}o16_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}o16_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}o16_tot_nc_n.csv"
}

h1_files = {
    'tot_cc': f"{csv_dir}h1_tot_cc.csv",
    'tot_nc': f"{csv_dir}h1_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}h1_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}h1_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}h1_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}h1_tot_nc_n.csv"
}

# Load data from CSV files
o16_data = {}
for name, file in o16_files.items():
    try:
        df = pd.read_csv(file)
        o16_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

h1_data = {}
for name, file in h1_files.items():
    try:
        df = pd.read_csv(file)
        h1_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Assume we're working with your event data


# Create Figure 1: Just the cross-sections+
plt.figure(figsize=(18, 12))

# Plot styles for different cross-section types
line_styles = {
    'tot_cc': {'style': '-', 'label': 'CC', 'color': 'b', 'width': 2},
    'tot_nc': {'style': '--', 'label': 'NC', 'color': 'b', 'width': 2},
    'tot_cc_p': {'style': ':', 'label': 'CC p', 'color': 'b', 'width': 1},
    'tot_cc_n': {'style': '-.', 'label': 'CC n', 'color': 'b', 'width': 1}
}

# Plot O16 cross-sections
for name, data in o16_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color=style['color'], linewidth=style['width'], 
                label=f'O16 {style["label"]}')

# Plot H1 cross-sections
for name, data in h1_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color='g', linewidth=style['width'], 
                label=f'H1 {style["label"]}')

# Calculate H2O cross-section for CC and NC
water_energies = np.linspace(10, 60, 100)
h2o_data = {}

for xsec_type in ['tot_cc', 'tot_nc']:
    if xsec_type in o16_data and xsec_type in h1_data:
        # Interpolate O16
        o16_interp = np.interp(water_energies, 
                               o16_data[xsec_type]['energy'], 
                               o16_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Interpolate H1
        h1_interp = np.interp(water_energies, 
                               h1_data[xsec_type]['energy'], 
                               h1_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Water = 1 oxygen + 2 hydrogen
        h2o_xsec = o16_interp + 2 * h1_interp
        
        # Store for later use
        h2o_data[xsec_type] = {
            'energy': water_energies,
            'xsec': h2o_xsec
        }
        
        # Plot H2O cross-section
        style = line_styles[xsec_type]
        # plt.plot(water_energies, h2o_xsec, 
        #         style['style'], color='r', linewidth=1, 
        #         label=f'H2O {style["label"]}')

# Calculate H2O total cross-section (CC + NC)
if 'tot_cc' in h2o_data and 'tot_nc' in h2o_data:
    h2o_total_xsec = h2o_data['tot_cc']['xsec'] + h2o_data['tot_nc']['xsec']
    h2o_data['tot'] = {
        'energy': water_energies,
        'xsec': h2o_total_xsec
    }
    # plt.plot(water_energies, h2o_total_xsec, 
    #         '-', color='r', linewidth=4, 
    #         label=f'H2O Total')
# Original scatter plot
plt.scatter(event_e, event_xsec,color='red', s=8, alpha=0.01)

# Finalize the plot
plt.xlabel('Energy (GeV)')
plt.ylabel('Cross-section (10^-38 cm^2)')
plt.title('GENIE Cross-sections')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(10, 60)
plt.ylim(0,900)
plt.show()

In [ ]:
# Filter events by interaction type and target
o16_cc_events = [event for event in parsed_events if 'CC' in event.event_descr and event.init_target_id == 1000080160]
h1_cc_events = [event for event in parsed_events if 'CC' in event.event_descr and event.init_target_id == 1000010010]

# Extract energy and cross-section data
o16_cc_energies = [event.init_inj_e for event in o16_cc_events]
o16_cc_xsec = [event.event_xsec * 1e38 for event in o16_cc_events]  # Convert to 10^-38 cm^2

# Now plot these against the reference curves
plt.figure(figsize=(12, 8))
plt.scatter(o16_cc_energies, o16_cc_xsec, color='red', s=8, alpha=0.1, label='O16 CC Simulation')
plt.plot(o16_data['tot_cc']['energy'], o16_data['tot_cc']['xsec'], '-', color='blue', linewidth=2, label='O16 CC Reference')
# Add similar for H1 events

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Paths to your CSV files
csv_dir = "/groups/icecube/jackp/tmp/"
o16_files = {
    'tot_cc': f"{csv_dir}o16_tot_cc.csv",
    'tot_nc': f"{csv_dir}o16_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}o16_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}o16_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}o16_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}o16_tot_nc_n.csv"
}

h1_files = {
    'tot_cc': f"{csv_dir}h1_tot_cc.csv",
    'tot_nc': f"{csv_dir}h1_tot_nc.csv",
    'tot_cc_p': f"{csv_dir}h1_tot_cc_p.csv",
    'tot_cc_n': f"{csv_dir}h1_tot_cc_n.csv",
    'tot_nc_p': f"{csv_dir}h1_tot_nc_p.csv",
    'tot_nc_n': f"{csv_dir}h1_tot_nc_n.csv"
}

# Load data from CSV files
o16_data = {}
for name, file in o16_files.items():
    try:
        df = pd.read_csv(file)
        o16_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

h1_data = {}
for name, file in h1_files.items():
    try:
        df = pd.read_csv(file)
        h1_data[name] = {
            'energy': df['energy'].values,
            'xsec': df['xsec'].values
        }
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Assume we're working with your event data
# Replace these with your actual event data
# event_e = your_energy_array
# event_xsec = your_xsec_array

# Create Figure 1: Just the cross-sections
plt.figure(figsize=(12, 8))

# Plot styles for different cross-section types
line_styles = {
    'tot_cc': {'style': '-', 'label': 'CC', 'color': 'b', 'width': 2},
    'tot_nc': {'style': '--', 'label': 'NC', 'color': 'b', 'width': 2},
    'tot_cc_p': {'style': ':', 'label': 'CC p', 'color': 'b', 'width': 1},
    'tot_cc_n': {'style': '-.', 'label': 'CC n', 'color': 'b', 'width': 1}
}

# Plot O16 cross-sections
for name, data in o16_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color=style['color'], linewidth=style['width'], 
                label=f'O16 {style["label"]}')

# Plot H1 cross-sections
for name, data in h1_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec'], 
                style['style'], color='g', linewidth=style['width'], 
                label=f'H1 {style["label"]}')

# Calculate H2O cross-section for CC and NC
water_energies = np.linspace(10, 60, 100)
h2o_data = {}

for xsec_type in ['tot_cc', 'tot_nc']:
    if xsec_type in o16_data and xsec_type in h1_data:
        # Interpolate O16
        o16_interp = np.interp(water_energies, 
                               o16_data[xsec_type]['energy'], 
                               o16_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Interpolate H1
        h1_interp = np.interp(water_energies, 
                               h1_data[xsec_type]['energy'], 
                               h1_data[xsec_type]['xsec'], 
                               left=0, right=0)
        
        # Water = 1 oxygen + 2 hydrogen
        h2o_xsec = o16_interp + 2 * h1_interp
        
        # Store for later use
        h2o_data[xsec_type] = {
            'energy': water_energies,
            'xsec': h2o_xsec
        }
        
        # Plot H2O cross-section
        style = line_styles[xsec_type]
        plt.plot(water_energies, h2o_xsec, 
                style['style'], color='r', linewidth=3, 
                label=f'H2O {style["label"]}')

# Calculate H2O total cross-section (CC + NC)
if 'tot_cc' in h2o_data and 'tot_nc' in h2o_data:
    h2o_total_xsec = h2o_data['tot_cc']['xsec'] + h2o_data['tot_nc']['xsec']
    h2o_data['tot'] = {
        'energy': water_energies,
        'xsec': h2o_total_xsec
    }
    plt.plot(water_energies, h2o_total_xsec, 
            '-', color='r', linewidth=4, 
            label=f'H2O Total')

# Finalize the plot
plt.xlabel('Energy (GeV)')
plt.ylabel('Cross-section (10^-38 cm^2)')
plt.title('GENIE Cross-sections')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(10, 60)
plt.show()

# If you have event data, create Figure 2: Comparison with event data
if 'event_e' in globals() and 'event_xsec' in globals():
    # Create binned data
    e_bins = np.linspace(10, 60, 51)
    bin_centers = (e_bins[:-1] + e_bins[1:]) / 2
    bin_indices = np.digitize(event_e, e_bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(e_bins)-2)
    
    mean_xsec_per_bin = []
    events_per_bin = []
    for i in range(len(e_bins)-1):
        mask = (bin_indices == i)
        count = np.sum(mask)
        events_per_bin.append(count)
        if count > 0:
            mean_xsec = np.mean(event_xsec[mask])
        else:
            mean_xsec = 0
        mean_xsec_per_bin.append(mean_xsec)
    
    # Create a figure with three subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 18))
    
    # Plot 1: Original scatter plot with reference cross-sections
    # First scatter the event data
    ax1.scatter(event_e, event_xsec, s=5, alpha=0.3, label='Event Data')
    
    # Add reference lines for H2O cross-sections
    ax1_ref = ax1.twinx()
    
    xsec_types = ['tot_cc', 'tot_nc', 'tot']
    styles = ['-', '--', '-.']
    labels = ['CC', 'NC', 'Total']
    
    for i, xsec_type in enumerate(xsec_types):
        if xsec_type in h2o_data:
            ax1_ref.plot(h2o_data[xsec_type]['energy'], 
                       h2o_data[xsec_type]['xsec'], 
                       styles[i], color='r', linewidth=2, 
                       label=f'H2O {labels[i]}')
    
    ax1.set_xlabel('Energy (GeV)')
    ax1.set_ylabel('Event XSec', color='blue')
    ax1_ref.set_ylabel('Reference XSec (10^-38 cm^2)', color='red')
    ax1.set_title('Event Cross-sections with Reference Lines')
    ax1.legend(loc='upper left')
    ax1_ref.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(10, 60)
    
    # Plot 2: Binned means with reference cross-sections
    ax2.bar(bin_centers, mean_xsec_per_bin, width=(e_bins[1]-e_bins[0]),
            align='center', alpha=0.5, edgecolor='black', label='Event XSec')
    
    # Create separate y-axis for reference data
    ax2_ref = ax2.twinx()
    
    # Interpolate H2O cross-sections to bin centers
    for i, xsec_type in enumerate(xsec_types):
        if xsec_type in h2o_data:
            h2o_interp = np.interp(bin_centers, 
                                 h2o_data[xsec_type]['energy'], 
                                 h2o_data[xsec_type]['xsec'], 
                                 left=0, right=0)
            
            ax2_ref.plot(bin_centers, h2o_interp, 
                      styles[i], color='r', linewidth=2, 
                      label=f'H2O {labels[i]}')
    
    ax2.set_xlabel('Energy (GeV)')
    ax2.set_ylabel('Event XSec', color='blue')
    ax2_ref.set_ylabel('Reference XSec (10^-38 cm^2)', color='red')
    
    ax2.legend(loc='upper left')
    ax2_ref.legend(loc='upper right')
    ax2.set_title('Comparison of Mean Event XSec and Reference Cross-sections')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Event distribution with expected distribution
    ax3.bar(bin_centers, events_per_bin, width=(e_bins[1]-e_bins[0]),
            align='center', alpha=0.7, edgecolor='black')
    ax3.set_xlabel('Energy (GeV)')
    ax3.set_ylabel('Number of Events')
    ax3.set_title('Event Count per Energy Bin')
    ax3.grid(True, alpha=0.3)
    
    # Calculate expected event distribution based on cross-section and E^-2 flux
    if np.sum(events_per_bin) > 0:
        for i, xsec_type in enumerate(xsec_types):
            if xsec_type in h2o_data:
                # Interpolate cross-section to bin centers
                h2o_interp = np.interp(bin_centers, 
                                     h2o_data[xsec_type]['energy'], 
                                     h2o_data[xsec_type]['xsec'], 
                                     left=0, right=0)
                
                # Apply E^-2 flux weighting
                flux_weight = 1.0 / (bin_centers ** 2)
                expected_distribution = h2o_interp * flux_weight
                
                # Scale to match total event count
                scaling = np.sum(events_per_bin) / np.sum(expected_distribution)
                expected_events = expected_distribution * scaling
                
                ax3.plot(bin_centers, expected_events, 
                       styles[i], color='r', linewidth=2, 
                       label=f'Expected from H2O {labels[i]} (E^-2 weighted)')
        
        ax3.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No event data available for comparison.")

In [140]:
high_cross =  final_parsed[final_parsed['event_xsec']>200]

In [ ]:
np.unique(high_cross['event_descr'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# # Paths to your CSV files
# csv_dir = "/groups/icecube/jackp/tmp/"
# o16_files = {
#     'tot_cc': f"{csv_dir}o16_tot_cc.csv",
#     'tot_nc': f"{csv_dir}o16_tot_nc.csv",
#     'tot_cc_p': f"{csv_dir}o16_tot_cc_p.csv",
#     'tot_cc_n': f"{csv_dir}o16_tot_cc_n.csv",
#     'tot_nc_p': f"{csv_dir}o16_tot_nc_p.csv",
#     'tot_nc_n': f"{csv_dir}o16_tot_nc_n.csv"
# }

# h1_files = {
#     'tot_cc': f"{csv_dir}h1_tot_cc.csv",
#     'tot_nc': f"{csv_dir}h1_tot_nc.csv",
#     'tot_cc_p': f"{csv_dir}h1_tot_cc_p.csv",
#     'tot_cc_n': f"{csv_dir}h1_tot_cc_n.csv",
#     'tot_nc_p': f"{csv_dir}h1_tot_nc_p.csv",
#     'tot_nc_n': f"{csv_dir}h1_tot_nc_n.csv"
# }

# # Load data from CSV files
# o16_data = {}
# for name, file in o16_files.items():
#     try:
#         df = pd.read_csv(file)
#         o16_data[name] = {
#             'energy': df['energy'].values,
#             'xsec': df['xsec'].values
#         }
#     except Exception as e:
#         print(f"Error loading {file}: {e}")

# h1_data = {}
# for name, file in h1_files.items():
#     try:
#         df = pd.read_csv(file)
#         h1_data[name] = {
#             'energy': df['energy'].values,
#             'xsec': df['xsec'].values
#         }
#     except Exception as e:
#         print(f"Error loading {file}: {e}")

# Assume we're working with your event data


# Create Figure 1: Just the cross-sections+
plt.figure(figsize=(18, 12))

# Plot styles for different cross-section types
line_styles = {
    'tot_cc': {'style': '-', 'label': 'CC', 'color': 'b', 'width': 2},
    'tot_nc': {'style': '--', 'label': 'NC', 'color': 'b', 'width': 2},
    'tot_cc_p': {'style': ':', 'label': 'CC p', 'color': 'b', 'width': 1},
    'tot_cc_n': {'style': '-.', 'label': 'CC n', 'color': 'b', 'width': 1}
}

# Plot O16 cross-sections
for name, data in o16_data.items():
    if name in line_styles:
        style = line_styles[name]
        plt.plot(data['energy'], data['xsec']*.52, 
                style['style'], color=style['color'], linewidth=style['width'], 
                label=f'O16 {style["label"]}')

# # Plot H1 cross-sections
# for name, data in h1_data.items():
#     if name in line_styles:
#         style = line_styles[name]
#         plt.plot(data['energy'], data['xsec'], 
#                 style['style'], color='g', linewidth=style['width'], 
#                 label=f'H1 {style["label"]}')

# Calculate H2O cross-section for CC and NC
water_energies = np.linspace(10, 60, 100)
h2o_data = {}

plt.scatter(event_e, event_xsec,color='red', s=8, alpha=0.01)

# Finalize the plot
plt.xlabel('Energy (GeV)')
plt.ylabel('Cross-section (10^-38 cm^2)')
plt.title('GENIE Cross-sections')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(10, 60)
plt.ylim(0,900)
plt.show()

In [ ]:
event_xse

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(event_e, event_dxsec, s=5)
plt.xlabel('energy (GeV)')
plt.ylabel('event_dxsec')
plt.ylim(0, 1000)
plt.plot()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the energy bins (same as your histogram)
e_bins = np.linspace(10, 60, 51)
bin_centers = (e_bins[:-1] + e_bins[1:]) / 2  # Calculate bin centers for plotting

# Group the event_dxsec by energy bins and calculate mean per bin
# First, digitize the energy values to assign each to a bin
bin_indices = np.digitize(event_e, e_bins) - 1  # -1 because digitize returns bin indices starting from 1
bin_indices = np.clip(bin_indices, 0, len(e_bins)-2)  # Ensure indices are within valid range

# Calculate mean event_dxsec for each bin
mean_xsec_per_bin = []
for i in range(len(e_bins)-1):
    # Find events in this bin
    mask = (bin_indices == i)
    if np.sum(mask) > 0:  # If bin has events
        mean_xsec = np.mean(event_dxsec[mask])
    else:
        mean_xsec = 0  # or np.nan if you prefer
    mean_xsec_per_bin.append(mean_xsec)

# Create two subplots - one for original scatter, one for binned means
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

# Original scatter plot
ax1.scatter(event_e, event_dxsec, s=5, alpha=0.5)
ax1.set_xlabel('Energy (GeV)')
ax1.set_ylabel('event_dxsec')
ax1.set_title('Original Scatter Plot')

# Binned means plot
ax2.bar(bin_centers, mean_xsec_per_bin, width=(e_bins[1]-e_bins[0]), 
       align='center', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Energy (GeV)')
ax2.set_ylabel('Mean event_dxsec')
ax2.set_title('Mean Cross-section per Energy Bin')

plt.tight_layout()
plt.show()